# 0. Imports and Toy Dataset Creation

In [ ]:
import math
import networkx as nx
import torch
from matplotlib import pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.metrics import (
    adjusted_mutual_info_score as AMI,
    adjusted_rand_score as ARI,
)

In [ ]:
import numpy as np
np.random.seed(42)

In [ ]:
X, y, centroids_true = make_blobs(
    n_samples=1000,
    centers=5,
    cluster_std=0.6,
    return_centers=True
)
plt.scatter(X[:,0], X[:,1], c=y)

# 1. First Attempt (Naive Implementation)

In [ ]:
class DBSCAN:
    def __init__(self, eps, minPts):
        self._eps = eps
        self._minPts = minPts

    def fit(self, X):
        pass

    def score(self, X, y):
        pass

    def predict(self, X):
        pass

    def _construct_graph(self, X):
        # store in a compact adj list
        # Va and Ea
        cardinality = len(X)
        graph = nx.Graph()
        graph.add_nodes_from(list(range(cardinality)), is_core=False)
        for i in range(cardinality):
            for j in range(i + 1, cardinality):
                if self._distance(X[i], X[j]) <= self._eps:
                    graph.add_edge(i, j)
            if graph.degree[i] >= self._minPts:
                graph.nodes[i]["is_core"] = True
        self._graph = graph

    @staticmethod
    def _distance(p1, p2):
        return np.linalg.norm(p1 - p2)
